<a href="https://colab.research.google.com/github/Danielmejiava/danielmejiava.github.io/blob/main/repaymentProgressCol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [16.0,8.0]

In [ ]:
from google.colab import drive
df = pd.read_csv('/content/ibrdStatement2025.csv')

In [ ]:
# First, filter the columns

# Then, filter the rows based on the countries
df= df[df['Country / Economy Code'].isin(['CO'])]

# Display the first few rows of the filtered DataFrame

filtered_df = df

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 283 entries, 0 to 9087
Data columns (total 33 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   End of Period                     283 non-null    object 
 1   Loan Number                       283 non-null    object 
 2   Region                            283 non-null    object 
 3   Country / Economy Code            283 non-null    object 
 4   Country / Economy                 283 non-null    object 
 5   Borrower                          283 non-null    object 
 6   Guarantor Country / Economy Code  283 non-null    object 
 7   Guarantor                         283 non-null    object 
 8   Loan Type                         283 non-null    object 
 9   Loan Status                       283 non-null    object 
 10  Interest Rate                     279 non-null    float64
 11  Currency of Commitment            0 non-null      float64
 12  Project ID  

In [ ]:
# Calculate Repayment Progress (%)
filtered_df['Repayment Progress (%)'] = (
    filtered_df['Repaid to IBRD (US$)'] / filtered_df['Original Principal Amount (US$)']
) * 100

# Replace infinite or missing repayment progress with NaN
filtered_df['Repayment Progress (%)'] = filtered_df['Repayment Progress (%)'].replace([float('inf'), -float('inf')], None)

# Drop rows where repayment progress is NaN (invalid values)
filtered_df = filtered_df.dropna(subset=['Repayment Progress (%)'])

# Group by Borrower and calculate the average repayment progress
repayment_by_borrower = filtered_df.groupby('Borrower', as_index=False).agg(
    average_repayment_progress=('Repayment Progress (%)', 'mean'),
    total_repaid=('Repaid to IBRD (US$)', 'sum'),
    total_principal=('Original Principal Amount (US$)', 'sum')
)

# Filter to ensure valid repayment progress
repayment_by_borrower = repayment_by_borrower[repayment_by_borrower['average_repayment_progress'] > 0]

# Select the 10 borrowers with the highest total principal amounts
top_10_borrowers = repayment_by_borrower.nlargest(10, 'total_principal')

# Display the filtered data
print("Top 10 Borrowers by Total Principal Amount:")
print(top_10_borrowers)

Top 10 Borrowers by Total Principal Amount:
                                    Borrower average_repayment_progress  \
46  MINISTERIO DE HACIENDA Y CREDITO PUBLICO                  63.857775   
4                      BANCO DE LA REPUBLICA                   94.55348   
34                                    ICETEX                  21.696772   
21              EMPRESA DE ENERGIA ELECTRICA                  94.042872   
40                INSTITUTO NACIONAL DE VIAS                  99.559128   
25             EMPRESAS PUBLICAS DE MEDELLIN                  90.473642   
41             INTERCONEXION ELECTRICA S. A.                  93.080192   
20  EMPRESA DE ACUEDUCTO Y ALCANTA DE BOGOTA                  93.283326   
11  Corporaci�n Aut�noma Regional deCundinam                  94.213366   
33   FONDO PARA LA RECONST. DEL EJE CAFETERO                      100.0   

    total_repaid  total_principal  
46  8.345150e+09     2.623606e+10  
4   9.602685e+08     1.019000e+09  
34  2.127455e+08     8

In [ ]:
import plotly.express as px

# Replace infinite or missing repayment progress with NaN
repayment_by_borrower['average_repayment_progress'].replace([float('inf'), -float('inf')], None, inplace=True)

# Round the repayment progress percentage to 2 decimal places
repayment_by_borrower['average_repayment_progress'] = repayment_by_borrower['average_repayment_progress'].round(2)

# **Filter the top 10 borrowers by total principal**
top_10_borrowers = repayment_by_borrower.nlargest(10, 'total_principal')

# Create an interactive scatter plot for the top 10 borrowers
fig = px.scatter(
    top_10_borrowers,  # Use the filtered DataFrame here
    x='total_principal',
    y='average_repayment_progress',
    size='total_repaid',  # Size of markers based on total repaid
    color='Borrower',  # Different color for each borrower
    hover_name='Borrower',  # Display borrower name on hover
    hover_data={
        'total_repaid': ':,.2f',  # Format total repaid with commas
        'average_repayment_progress': ':.2f',  # Format percentage with 2 decimals
        'total_principal': ':,.2f'  # Format total principal with commas
    },
    labels={
        'total_principal': 'Total Principal (US$)',
        'average_repayment_progress': 'Repayment Progress (%)'
    },
    title='Interactive Scatter Plot: Repayment Progress vs Total Principal (Top 10 Borrowers)'
)

# Update layout for better readability
fig.update_layout(
    xaxis=dict(title='Total Principal (US$)', tickformat=","),
    yaxis=dict(title='Repayment Progress (%)'),
    title=dict(x=0.5),  # Center the title
    template='plotly_white'  # Use a clean style
)

# Show the interactive plot
fig.show()

<ipython-input-21-6523896de68f>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  repayment_by_borrower['average_repayment_progress'].replace([float('inf'), -float('inf')], None, inplace=True)


The visual addresses the research question by effectively showing the relationship between Total Principal (US$) borrowed and Repayment Progress (%) for different borrowers. By plotting borrowers as data points, the graph highlights variations in loan amounts (represented by the x-axis) and repayment performance (y-axis). The size of each point further conveys the relative weight of the loans in the total dataset, helping to identify which borrowers contribute significantly to the portfolio.

For example, Ministerio de Hacienda y Crédito Público, represented by the large pink point, stands out as a borrower with the largest total principal amount and almost 100% repayment progress. This insight emphasizes its significant role in the overall portfolio and its strong repayment performance. Additionally, the scatterplot identifies outliers, such as borrowers with lower repayment progress or smaller loan amounts, offering a comprehensive overview for evaluating borrower performance and portfolio health.